In [1]:
%matplotlib qt
import scipy
from scipy.io import wavfile
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import signal
import osascript
from gtts import gTTS 
import os 
import pyaudio
import wave
import keyboard  # using module keyboard
import soundfile as sf
import math
import pyloudnorm as pyln
from sys import byteorder
from array import array
from struct import pack
import librosa
from scipy.signal import butter, sosfiltfilt
import python_speech_features
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import train_test_split
from ipynb.fs.full.Pitch_vector import get_pitch_stats
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics 
import time
import  ipynb.fs.full.concat_project2 as emotex_lib
from sklearn.model_selection import KFold
from keras.optimizers import Adadelta
from keras.layers import Conv2D
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
import keras as keras
from  conch.analysis.formants import lpc

Using TensorFlow backend.


In [2]:
X, fs, x_size = emotex_lib.data_extraction_RAVDESS('../../RAVDESS')

KeyboardInterrupt: 

In [ ]:
Y = emotex_lib.emotion_extraction_RAVDESS('../../RAVDESS',1380)

In [3]:
np.sum(Y[:,5])

88.0

In [ ]:
X_train, X_test, y_train, y_test = emotex_lib.x_y_split(X, fs, Y)

In [23]:
def MLPalgorithm(X_train, X_test, y_train, y_test):
    mlp = MLPClassifier(max_iter=10000)
    mlp.fit(X_train,y_train)
    predictions = mlp.predict(X_train)
    print(classification_report(y_train,predictions))
    predictions = mlp.predict(X_test)
    print(classification_report(y_test,predictions))

In [ ]:
def complex_cnn(X_train_test_splitain, X_test, y_train, y_test):
    model = Sequential()
    model.add(Conv1D(256, 5,padding='same', input_shape=(X_train[1],1))) #1
    model.add(Activation('relu'))
    model.add(Conv1D(128, 5,padding='same')) #2
    model.add(Activation('relu'))
    model.add(Dropout(0.1))
    model.add(MaxPooling1D(pool_size=(8)))
    model.add(Conv1D(128, 5,padding='same')) #3
    model.add(Activation('relu'))
    #model.add(Conv1D(128, 5,padding='same')) #4
    #model.add(Activation('relu'))
    #model.add(Conv1D(128, 5,padding='same')) #5
    #model.add(Activation('relu'))
    #model.add(Dropout(0.2))
    model.add(Conv1D(128, 5,padding='same')) #6
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(10)) #7
    model.add(Activation('softmax'))
    opt = keras.optimizers.rmsprop(lr=0.00001, decay=1e-6)
    model.compile(loss='categorical_crossentropy', optimizer=opt,metrics=['accuracy'])
    cnnhistory=model.fit(X_train, y_train, validation_data=(X_test, y_test))
    

In [24]:
MLPalgorithm(X_train, X_test, y_train, y_test)

              precision    recall  f1-score   support

           0       0.32      0.14      0.20        69
           1       0.71      0.35      0.47       150
           2       0.28      0.15      0.19       149
           3       0.64      0.05      0.09       138
           4       0.58      0.40      0.47       149
           5       0.20      0.64      0.31       150
           6       0.47      0.25      0.33       147
           7       0.37      0.48      0.42       152

   micro avg       0.34      0.32      0.33      1104
   macro avg       0.45      0.31      0.31      1104
weighted avg       0.45      0.32      0.32      1104
 samples avg       0.25      0.32      0.27      1104

              precision    recall  f1-score   support

           0       0.40      0.26      0.32        23
           1       0.45      0.26      0.33        34
           2       0.35      0.17      0.23        35
           3       0.25      0.02      0.04        46
           4       0.20 

/usr/local/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


In [ ]:
np.save("DELETE", (X,fs,x_size))

In [2]:
(X,fs,x_size) = np.load("DELETE.npy",  allow_pickle=True)

In [3]:
Y = emotex_lib.emotion_extraction_RAVDESS('../../RAVDESS',x_size)

In [ ]:
X_train, X_test, y_train, y_test =np.load("RAVDESS_X_Y_train_test.npy",  allow_pickle=True)

In [ ]:
X_train, X_test, y_train, y_test = emotex_lib.x_y_split(X, fs, Y)

In [7]:
np.save("RAVDESS_X_Y_train_test", (X_train, X_test, y_train, y_test))

In [ ]:
def emotion_extraction_RAVDESS(folder_location, number_examples):
    nu_emotion = 8
    y = np.zeros(shape=(nu_emotion, number_examples))
    counter = 0
    directory = os.fsencode(folder_location)
    for file_name in os.listdir(directory):
        if  str(file_name) != "b'.DS_Store'":
            emotion = str(file_name)[2:-1]
            i = (emotion.split('-')[2])
            if i == '01': # neutral 
                y[0][counter] = 1
            elif i == '02': #calm
                 y[1][counter] = 1
            elif i == '03': #hahppy 
                 y[2][counter] = 1
            elif i == '04': #sad
                 y[3][counter] = 1
            elif i == '05': #angry
                 y[4][counter] = 1
            elif i == '06': #fearful
                 y[5][counter] = 1
            elif i == '07': #disgust
                 y[6][counter] = 1
            elif i == '08': #surprised
                 y[7][counter] = 1
            else:
                print(i)
                break
            counter +=1
    y = np.transpose(y)
    return y

In [ ]:
# normalize dataset and test set
# dropout regularization
# L2 regularization
# early stopping
#  torrent the dataset

In [36]:
MLPalgorithm(X_train, X_test, y_train, y_test)

              precision    recall  f1-score   support

           0       1.00      0.01      0.03        69
           1       0.87      0.32      0.47       150
           2       1.00      0.02      0.04       149
           3       0.81      0.16      0.27       138
           4       0.50      0.03      0.06       149
           5       0.91      0.07      0.12       150
           6       0.80      0.05      0.10       147
           7       0.78      0.05      0.09       152

   micro avg       0.83      0.09      0.17      1104
   macro avg       0.83      0.09      0.15      1104
weighted avg       0.82      0.09      0.15      1104
 samples avg       0.09      0.09      0.09      1104

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        23
           1       0.53      0.24      0.33        34
           2       0.00      0.00      0.00        35
           3       0.21      0.07      0.10        46
           4       0.00 

/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


In [35]:
def MLPalgorithm(X_train, X_test, y_train, y_test):
    mlp = MLPClassifier(max_iter=15000)
    mlp.fit(X_train,y_train)
    predictions = mlp.predict(X_train)

    print(classification_report(y_train,predictions))
    predictions = mlp.predict(X_test)
    print(classification_report(y_test,predictions))
seed = 20
num_labels =15
num_features = X_train.shape[1]
# build model
def baseline_model():
    model = Sequential()
    model.add(Dense(32, input_dim=num_features, activation='softmax'))
    model.add(Dense(30, activation='softmax'))
    model.add(Dense(output_dim = num_labels, activation='softmax'))
    model.compile(loss='categorical_crossentropy',
                  optimizer=Adadelta(), metrics=['categorical_accuracy'])
    return model
def big_boy_CNN():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(15, activation='softmax'))
    model.compile(
            optimizer='adam',
            loss='categorical_crossentropy',
            metrics=['accuracy'])
    return model

def complex_layers():
    model = Sequential()
    model.add(Conv1D(256, 5,padding='same', input_shape=( 11,13, 1))) #1
    model.add(Activation('relu'))
    model.add(Activation('relu'))
    model.add(Conv1D(128, 5,padding='same')) #2
    model.add(Activation('relu'))
    model.add(Dropout(0.1))
    model.add(MaxPooling1D(pool_size=(8)))
    model.add(Conv1D(128, 5,padding='same')) #3
    model.add(Activation('relu'))
    model.add(Conv1D(128, 5,padding='same')) #4
    model.add(Activation('relu'))
    model.add(Conv1D(128, 5,padding='same')) #5
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Conv1D(128, 5,padding='same')) #6
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(10)) #7
    model.add(Activation('softmax'))
    opt = keras.optimizers.rmsprop(lr=0.00001, decay=1e-6)
    model.compile(loss='categorical_crossentropy', optimizer=opt,metrics=['accuracy'])
    return model
def easy_NN():
    mlp = MLPClassifier(max_iter=15000)
    return mlp

In [ ]:
np_data = X
MFCC2 = []
for one_sound in np_data:
    one_sound = np.asarray(one_sound)
    MFCC2.append(python_speech_features.base.mfcc(one_sound, samplerate=fs, 
                                 winlen=0.025, winstep=0.01, numcep=13, 
                                 nfilt=26, nfft=1200).T)
MFCC3 = []
cached_variables = []
for one_point in MFCC2:
    cache_grad = (np.gradient(one_point, axis = 1))
    cached_variables.append([np.mean(one_point, axis = 1), np.median(one_point, axis = 1),
                             np.var(one_point, axis = 1), 
                       np.min(one_point, axis = 1), np.max(one_point, axis = 1), 
                             np.mean(cache_grad, axis = 1), np.var(cache_grad, axis = 1)])
return cached_variables

In [4]:
plt.plot([1,2,4,5])

In [38]:
np.mean(one_point, axis = 1).shape, np.median(one_point, axis = 1).shape,np.var(one_point, axis = 1).shape, np.min(one_point, axis = 1).shape, np.max(one_point, axis = 1).shape, np.mean(cache_grad, axis = 1).shape, np.var(cache_grad, axis = 1).shape


NameError: name 'one_point' is not defined

In [ ]:
np.mean(please.T, axis = 1).shape

In [ ]:
len(MFCC3)

In [ ]:
np.mean(MFCC2[0], axis=1).shape

In [ ]:
len(np.gradient(MFCC2[0], axis = 1)), len(np.gradient(MFCC2[0], axis = 1)[0])